In [16]:
import pandas as pd

# Load the data from the uploaded CSV file
file_path = 'https://raw.githubusercontent.com/lilfakeS/PPP/main/PAPERLIVE%20%E5%B8%82%E5%9C%BA%E4%BB%B7%E6%A0%BC%E6%9C%80%E6%96%B0%E7%89%88_2023.csv'
paper_data = pd.read_csv(file_path)

# Display the first few rows of the data to understand its structure
paper_data.head()

,ID,Weight,Length,Width,Price,Price_1,Weight_1
0,1,135.0,900.0,640.0,1.94,0.12,9.028125
1,2,170.0,597.0,889.0,2.26,0.14,11.368750
2,3,170.0,889.0,635.0,2.40,0.15,11.368750
3,4,95.0,787.0,1092.0,1.55,0.10,6.353125
4,5,95.0,889.0,1194.0,1.92,0.12,6.353125


In [2]:
# Load the data from the uploaded CSV file
trans_file_path = 'https://raw.githubusercontent.com/lilfakeS/PPP/main/Trans.csv'
trans_data = pd.read_csv(trans_file_path)

# Display the first few rows of the data to understand its structure
trans_data.head()

,KG,RMB,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,0.5,228.6,NaN,NaN,NaN,NaN
1,1,257.4,NaN,NaN,NaN,NaN
2,1.5,286,NaN,NaN,NaN,NaN
3,2,314. 6,NaN,NaN,NaN,NaN
4,2.5,343. 3,NaN,NaN,NaN,NaN


In [3]:

# Remove any whitespace from the 'RMB' column
trans_data['RMB'] = trans_data['RMB'].astype(str).str.replace(" ", "")

# Convert the 'RMB' column to numeric values
trans_data['RMB'] = pd.to_numeric(trans_data['RMB'], errors='coerce')

# Drop the empty and unnecessary columns
trans_data = trans_data[['KG', 'RMB']].dropna()

# Convert 'KG' to numeric as well, just in case it's read as a string
trans_data['KG'] = pd.to_numeric(trans_data['KG'], errors='coerce')

# Display the cleaned data
trans_data.head()

,KG,RMB
0,0.5,228.6
1,1.0,257.4
2,1.5,286.0
3,2.0,314.6
4,2.5,343.3


In [18]:
# Correcting the column name and categorizing the items
paper_data['Class'] = paper_data[' Price '].apply(lambda x: 'A' if x < 6 else 'B')

# Display the updated dataframe to verify the classification
paper_data.head()


,ID,Weight,Length,Width,Price,Price_1,Weight_1,Class
0,1,135.0,900.0,640.0,1.94,0.12,9.028125,A
1,2,170.0,597.0,889.0,2.26,0.14,11.368750,A
2,3,170.0,889.0,635.0,2.40,0.15,11.368750,A
3,4,95.0,787.0,1092.0,1.55,0.10,6.353125,A
4,5,95.0,889.0,1194.0,1.92,0.12,6.353125,A


In [19]:
# Defining discount percentages for each channel
discounts = {
    'distributor': 0.40,  # 40% discount
    'dealer': 0.25,       # Average of 20-30% discount, using 25% as a midpoint
    'retailer': 0.10       # 10% discount
}

# Applying discounts to calculate selling prices for each channel
for channel, discount in discounts.items():
    discounted_price_column = f'Price_{channel}'
    paper_data[discounted_price_column] = paper_data['Price_1'] * (1 - discount)

# Display the dataframe with new columns for discounted prices
paper_data[['Price_1', 'Price_distributor', 'Price_dealer', 'Price_retailer']].head()


,Price_1,Price_distributor,Price_dealer,Price_retailer
0,0.12,0.072,0.0900,0.108
1,0.14,0.084,0.1050,0.126
2,0.15,0.090,0.1125,0.135
3,0.10,0.060,0.0750,0.090
4,0.12,0.072,0.0900,0.108


In [65]:
# Total estimated annual revenue in RMB
total_revenue = 320000/2.5

# Sales distribution by class
sales_distribution = {
    'A': 0.45,  # 45% of sales are A class products
    'B': 0.55   # 55% of sales are B class products
}

# Sales distribution by channel
channel_distribution = {
    'distributor': 0.60,  # 60% of sales go to distributors
    'dealer': 0.25,       # 25% of sales go to dealers
    'retailer': 0.10      # 10% of sales go to retailers
    # Remaining 5% can be attributed to direct sales or other channels
}

# Calculating total sales volume for each class
total_sales_volume = {
    'A': total_revenue * sales_distribution['A'],
    'B': total_revenue * sales_distribution['B']
}

# Allocating sales volume to each channel based on their distribution
channel_revenue = {}
for channel in channel_distribution:
    revenue_A = total_sales_volume['A'] * channel_distribution[channel]
    revenue_B = total_sales_volume['B'] * channel_distribution[channel]
    channel_revenue[channel] = revenue_A + revenue_B

channel_revenue



{'distributor': 76800.0, 'dealer': 32000.0, 'retailer': 12800.0}

In [66]:
# Calculating the average selling price for each class and channel
average_prices = {}
for channel in channel_distribution:
    # Calculating average price for each class
    for class_type in ['A', 'B']:
        discounted_price_column = f'Price_{channel}'
        # Filtering data based on class and calculating the average
        average_price = paper_data[paper_data['Class'] == class_type][discounted_price_column].mean()
        average_prices[(channel, class_type)] = average_price

# Calculating the weighted average selling price for the total sales
weighted_avg_price = sum(
    [average_prices[(channel, class_type)] * sales_distribution[class_type] * channel_distribution[channel] 
     for channel in channel_distribution for class_type in ['A', 'B']]
)

# Calculating total sales volume
total_sales_volume = total_revenue / weighted_avg_price

# Calculating sales volume for each channel
channel_sales_volume = {channel: channel_revenue[channel] / average_prices[(channel, class_type)]
                        for channel in channel_distribution for class_type in ['A', 'B']}

total_sales_volume, channel_sales_volume





(248329.45163895932,
 {'distributor': 146687.35824280768,
  'dealer': 48895.7860809359,
  'retailer': 16298.595360311963})

In [67]:
# Recalculating the weighted average selling prices for each channel
weighted_avg_prices_channel = {}
for channel in channel_distribution:
    weighted_avg_price_channel = sum(
        average_prices[(channel, class_type)] * sales_distribution[class_type] 
        for class_type in ['A', 'B']
    )
    weighted_avg_prices_channel[channel] = weighted_avg_price_channel

# Calculating the overall weighted average selling price
overall_weighted_avg_price = sum(
    weighted_avg_prices_channel[channel] * channel_distribution[channel] 
    for channel in channel_distribution
)

# Recalculating total sales volume
total_sales_volume_corrected = total_revenue / overall_weighted_avg_price

# Recalculating sales volume for each channel
channel_sales_volume_corrected = {
    channel: channel_revenue[channel] / weighted_avg_prices_channel[channel] 
    for channel in channel_distribution
}

total_sales_volume_corrected, channel_sales_volume_corrected



(248329.45163895932,
 {'distributor': 158310.02541983654,
  'dealer': 52770.008473278845,
  'retailer': 17590.00282442628})

In [68]:
# Recalculating the total sales volume
total_sales_volume_recalculated = total_revenue / overall_weighted_avg_price

# Distributing the total sales volume among the channels
channel_sales_volume_recalculated = {
    channel: total_sales_volume_recalculated * channel_distribution[channel] 
    for channel in channel_distribution
}

total_sales_volume_recalculated, channel_sales_volume_recalculated



(248329.45163895932,
 {'distributor': 148997.67098337557,
  'dealer': 62082.36290973983,
  'retailer': 24832.945163895933})

In [69]:
# Calculating the average purchase price (assuming Price_1 as a reference)
average_purchase_price = paper_data['Price_1'].mean()

# Calculating total purchase cost
total_purchase_cost = average_purchase_price * total_sales_volume_recalculated

# Calculating purchase cost for each channel
channel_purchase_cost = {
    channel: average_purchase_price * volume 
    for channel, volume in channel_sales_volume_recalculated.items()
}

average_purchase_price, total_purchase_cost, channel_purchase_cost



(0.8358578052550231,
 207568.11042712393,
 {'distributor': 124540.86625627434,
  'dealer': 51892.02760678098,
  'retailer': 20756.811042712394})

In [70]:
# Calculating the average weight per unit
average_weight_per_unit = paper_data['Weight_1'].mean()

# Calculating total weight of the goods
total_weight = average_weight_per_unit * total_sales_volume_recalculated/1000

average_weight_per_unit, total_weight



(13.783255781119964, 3422.7883499250365)

In [71]:
total_cost = total_weight* 65 + total_purchase_cost
print(total_cost)

430049.3531722513


In [72]:
discount_rate = (total_cost-320000) / 320000
print(discount_rate)

0.34390422866328535
